## Use Decision Tree for forcast IMOEX price
### Vol 4
#### 17.06.19

In [1]:
# connecting nessesary paceges
from pandas import read_csv as read
from pandas import DataFrame as DF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from IPython.display import SVG
from IPython.display import display
from graphviz import Source
from IPython.display import HTML
style = "<style>svg{width:80% !important;height:80% !important;}</style>"
HTML(style)

In [2]:
# In this part used shares which total weight amount more what 70% of index IMOEX
# GAZP (17,68%), LKOH (30,59%), SBER (44,79%), NVTK (50,61%), GMKN (56,56%), ROSN (61,09), YNDX (64,31%), TATN (68,46), SNGSP (70,6%)
data = read('https://raw.githubusercontent.com/Aurilino/Python/master/Decision%20Tree%20IMOEX/Index_min_large.txt')
data.index = range(len(data))
data['MOEX_1'] = data.MOEX.shift(-1) # let's try to predict one step forward
data['MOEX_1'] = DF(data['MOEX_1']).apply(lambda x: np.log(x) - np.log(x).shift(1))
data.dropna(inplace=True)
data['MOEX_1'] = data['MOEX_1'].apply(lambda x: 1 if x > 0 else 0)
data = data.drop(['Time', 'MOEX'], axis=1)
data.head()

c:\python37-32\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


,LKOH,SBER,GAZP,NVTK,GMKN,ROSN,YNDX,TATN,SNGSP,MOEX_1
1,5194.5,241.11,234.15,1308.4,14600.0,415.85,2483.4,765.6,41.165,1
2,5199.0,241.20,234.42,1312.6,14590.0,416.45,2482.8,767.8,41.150,1
3,5200.5,241.25,234.42,1321.0,14614.0,416.30,2485.2,767.8,41.155,1
4,5204.5,241.19,234.60,1321.4,14620.0,416.00,2484.2,766.4,41.150,1
5,5213.5,241.10,234.84,1320.4,14652.0,416.60,2481.6,768.6,41.185,0


## Let's look at the results with different length of the calculation window

In [3]:
for step in range(10, 160, 10):
    clf = DecisionTreeClassifier(criterion='entropy', max_depth=5)
    start = 0
    finish = start + step
    result = []
    fact = []
    while finish <= len(data)-3:
        train = data.iloc[start:finish, :]
        x = train.drop('MOEX_1', axis=1)
        y = train.MOEX_1
        clf.fit(x, y)
        x_test = data.iloc[finish+1:finish+2, :].drop('MOEX_1',axis=1)
        result.append(int(clf.predict(x_test)))
        fact.append(int(data.iloc[finish+1:finish+2, :].MOEX_1))
        start += 1
        finish += 1
    ans = DF({'result': result, 'fact': fact})
    ans['diff'] = ans['result'] == ans['fact']
    # get share of mistakes
    print('Share of coincidence: ', round(sum(ans['diff']) / len(ans) * 100, 2), '%',sep='')

Share of coincidence: 49.72%
Share of coincidence: 54.05%
Share of coincidence: 52.38%
Share of coincidence: 50.61%
Share of coincidence: 52.85%
Share of coincidence: 51.63%
Share of coincidence: 51.35%
Share of coincidence: 48.25%
Share of coincidence: 51.09%
Share of coincidence: 51.13%
Share of coincidence: 54.69%
Share of coincidence: 55.28%
Share of coincidence: 55.08%
Share of coincidence: 58.41%
Share of coincidence: 59.72%


## Summary
### As you can see, as the calculation window increases, the proportion of correctly predicted values increases.

## Note:
### For more detailed verification of the strategy, you should increase the number of verification bits.